In [ ]:
import librosa #https://librosa.org/
import librosa.display
import librosa.beat
import sounddevice as sd  #https://anaconda.org/conda-forge/python-sounddevice
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import scipy.stats as scs
import scipy.fft as scf
import scipy.signal as scsg
from sys import getsizeof as sizeof

sr = 22050
mono = True
warnings.filterwarnings("ignore")

In [ ]:
fName = "Queries/MT0000202045.mp3"

In [ ]:
y, fs = librosa.load(fName, sr=sr, mono = mono)
print(y.shape)
print(fs)

In [ ]:
sd.play(y, sr, blocking=False)

In [ ]:
plt.figure()
librosa.display.waveshow(y)

In [ ]:
Y = np.abs(librosa.stft(y))
Ydb = librosa.amplitude_to_db(Y, ref=np.max)
fig, ax = plt.subplots()
img = librosa.display.specshow(Ydb, y_axis='linear', x_axis='time', ax=ax)
ax.set_title('Power spectrogram')
fig.colorbar(img, ax=ax, format="%+2.0f dB")

In [ ]:
rms = librosa.feature.rms(y = y)
rms = rms[0, :]
print(rms.shape)
times = librosa.times_like(rms)
plt.figure(), plt.plot(times, rms)
plt.xlabel('Time (s)')
plt.title('RMS')

# Semana 2

In [ ]:
features = pd.read_csv("dataset/Features - Audio MER/features.csv", delimiter=';')
features

In [ ]:
top_100 = pd.read_csv("dataset/Features - Audio MER/top100_features.csv")
top_100.head()

In [ ]:
extracted_features = pd.read_csv("exercise2_features_all_sorted.csv", sep = ";", header = None)
extracted_features.head()

In [ ]:
def min_max_scale(y: np.array) -> np.float32:
    min_v = y.min()
    max_v = y.max()
    if max_v - min_v == 0:
        return 0
    return (y - min_v) / (max_v - min_v)

In [ ]:
top_100_normalized = np.apply_along_axis(min_max_scale, 0, top_100.to_numpy()[:,1:-1])
np.savetxt(f"top_100_features_normalized.csv", top_100_normalized, delimiter=";")

In [ ]:
extracted_features_normalized = np.apply_along_axis(min_max_scale, 0, extracted_features.to_numpy())
np.savetxt("exercise2_features_normalized.csv", extracted_features_normalized, delimiter=";")

In [ ]:
extracted_features_normalized.min()

In [ ]:
#Receives uniform signal and extracts features
def extract_features(signal : np.array):
    mean = signal.mean()
    stdDev = signal.std()
    skewness = scs.skew(signal)
    kurtosis = scs.kurtosis(signal)
    median = np.median(signal)
    max_value = signal.max()
    min_value = signal.min()
    return (mean, stdDev, skewness, kurtosis, median, max_value, min_value)

In [ ]:
path = "dataset/MER_audio_taffc_dataset/Q"
#910x190
feature_matrix = np.zeros((910, 190))
index=0
for i in range(1,5):
    for audio in os.listdir(f"{path}{i}"):
        print(index)
        y, fs = librosa.load(f"{path}{i}/{audio}", sr=sr, mono = mono)
        #Spectral features extraction
        mfcc = np.apply_along_axis(extract_features, 1, librosa.feature.mfcc(y=y, n_mfcc=13)).flatten()
        spc_centroid= np.apply_along_axis(extract_features, 1, librosa.feature.spectral_centroid(y=y)).flatten()
        spc_bdwth = np.apply_along_axis(extract_features, 1, librosa.feature.spectral_bandwidth(y=y)).flatten()
        spc_contrast = np.apply_along_axis(extract_features, 1, librosa.feature.spectral_contrast(y=y)).flatten()
        spc_flatness = np.apply_along_axis(extract_features, 1, librosa.feature.spectral_flatness(y=y)).flatten()
        spc_rollof = np.apply_along_axis(extract_features, 1, librosa.feature.spectral_rolloff(y=y)).flatten()
        f0 = np.apply_along_axis(extract_features, 0, librosa.yin(y, 20, 11025))
        f0[f0==11025] = 0;
        rms = np.apply_along_axis(extract_features, 1, librosa.feature.rms(y=y)).flatten()
        zcr = np.apply_along_axis(extract_features, 1, librosa.feature.zero_crossing_rate(y=y)).flatten()
        tempo = librosa.beat.tempo(y=y)
        
        feature_vector = np.concatenate((mfcc,spc_centroid, spc_bdwth, spc_contrast, spc_flatness, spc_rollof,
                                        f0, rms, zcr, tempo))
        
        feature_matrix[index]=feature_vector;
        index+=1;

np.apply_along_axis(librosa.util.normalize, 1, feature_matrix)
feature_matrix.tofile("exercise2_features.csv", sep = ";")
del feature_matrix

In [ ]:
def vectorDiff(music, feature_matrix):
    features = np.genfromtxt("top_100_features_normalized.csv")
    musics = np.zeros(900,2)
    music_sorted = np.zeros(20)
    for i in range(900):
        difference = music - feature_matrix[i]
        if(difference!=0):
            musics[i][0] = i
            musics[i][1] = difference
        sort = musics[musics[:, 1].argsort()]
        
    for i in range(20):
        music_sorted[i] = features[sort[i],[0],0]
        
    np.savetxt("best_musics_recomendations", music_sorted, delimiter=";")

In [ ]:
q1 = os.listdir("dataset/MER_audio_taffc_dataset/Q1/")
q2 = os.listdir("dataset/MER_audio_taffc_dataset/Q2/")
q3 = os.listdir("dataset/MER_audio_taffc_dataset/Q3/")
q4 = os.listdir("dataset/MER_audio_taffc_dataset/Q4/")
qs = q1 + q2 + q3 + q4

In [ ]:
df = pd.read_csv("exercise2_features.csv", sep=";", header=None)

In [ ]:
df.index = qs

In [ ]:
df.index

In [ ]:
df = df.sort_index()

In [ ]:
df.to_csv("exercise2_features_all_sorted.csv", )

In [ ]:
np.savetxt("exercise2_features_all_sorted.csv", df.to_numpy(), delimiter=";")

In [ ]:
def get_songs():
    list_songs = []
    for i in range(1, 5):
        list_songs += os.listdir(f"dataset/MER_audio_taffc_dataset/Q{i}")

    return list_songs

In [ ]:
files = [
    "extracted_cosine.csv",
    "extracted_euclidean.csv",
    "extracted_manhattan.csv",
    "top100_cosine.csv",
    "top100_euclidean.csv",
    "top100_manhattan.csv",
]

queries = [
    "MT0000202045.mp3",
    "MT0000379144.mp3",
    "MT0000414517.mp3",
    "MT0000956340.mp3",
]

list_songs = np.array(sorted(get_songs()))

for file in files:
    features = np.loadtxt(f"similarity_matrix/{file}", delimiter=";")

    for query in queries:
        song = np.where(query == list_songs)[0][0]
        rankings = np.argsort(features[song])[0:20]

        print(f"{query[:-4]}/{file}")
        np.savetxt(f"similarity_matrix/queries/{query[:-4]}/{file}", list_songs[rankings], fmt = "%s",delimiter=";")
